In [1]:
import numpy as np

In [2]:
class Node:
    def __init__(self, key):
        self.key = key
        self.edges = None
        self.next = None
        self.color = 'white' 
        
    def __repr__(self):
        return f'Node "{self.key}"'
    
    def __iter__(self):
        item = self.edges
        while item is not None:
            yield item
            item = item.next
            
    def get_edges(self):
        return [item for item in self]
    
        
class Edge:
    def __init__(self, node_in, weight=None, key=None):
        self.next = None
        self.node_in = node_in
        self.weight = weight
        self.key = key
        
    def __repr__(self):
        return f'Edge "key-{self.key}, weight-{self.weight}, to-{self.node_in}"'
        
class Graph:
    def __init__(self, node=None, adges=Node):
        self.head_node = node
        
    def __iter__(self):
        item = self.head_node
        while item is not None:
            yield item
            item = item.next
            
    def add_node(self, node):
        if self.head_node is None:
            self.head_node = node
            return
        last_node = self.head_node
        while last_node.next:
            last_node = last_node.next
        last_node.next = node
    
    
    def search_node(self, key):
        if self.head_node is None:
            return
        next_node = self.head_node
        if next_node.key == key:
            return next_node
        while next_node.next:
            next_node = next_node.next
            if next_node.key == key:
                return next_node
    
    
    def add_edge(self, node_key_out, node_key_in,  weight=None, key=None):
        node_out = self.search_node(node_key_out)
        node_in = self.search_node(node_key_in)
        edge = Edge(node_in, weight, key)
        if not node_out or not node_in:
            print('Не найдена вершина')
            return
        if node_out.edges is None:
            node_out.edges = edge
            return
        next_edge = node_out.edges
        while next_edge.next:
            next_edge = next_edge.next
        next_edge.next = edge
        
    def remove_edge_out(self, node_key_out, node_key_in):
        node_out = self.search_node(node_key_out)
#         node_in = self.search_node(node_key_in)
        if not node_out.edges:
            return
        next_edge = node_out.edges
        if next_edge.node_in.key == node_key_in:
            node_out.edges = next_edge.next
            return
        while next_edge.next:
            if next_edge.next.node_in.key == node_key_in:
                next_edge.next = next_edge.next.next
                return
            next_edge = next_edge.next
            
    def remove_node(self, node_key):
        self.remove_edges_in(node_key)
        
        next_node = self.head_node
        if next_node.key == node_key:
            self.head_node = next_node.next
            return
        while next_node:
            if next_node.next.key == node_key:
                next_node.next = next_node.next.next
            next_node = next_node.next
        
        
    def del_edges(self, node, node_key_in):
        if not node.edges:
            return
        edge = node.edges
        if edge.node_in.key == node_key_in:
            node.edges = edge.next
        while edge.next:
            if edge.next.node_in.key == node_key_in:
                edge.next = edge.next.next
                continue
            edge = edge.next
    
    def remove_edges_in(self, node_key_in):
        if not self.head_node:
            return       
        node = self.head_node
        while node:
            self.del_edges(node, node_key_in)            
            node = node.next
            
    def get_adjacent_nodes(self, node):
        return [item.node_in for item in node]
    
    def refresh_color(self):
        for item in self:
            item.color = 'white'
            
    def nodes_key_list(self):
        return [item.key for item in self]
    
    def nodes_list(self):
        return [item for item in self]
    


In [3]:
g = Graph()

g.add_node(Node('1'))
g.add_node(Node('2'))
g.add_node(Node('3'))
g.add_node(Node('4'))
g.add_node(Node('5'))
g.add_node(Node('6'))


g.add_edge('1','6', 14)
g.add_edge('1','3', 9)
g.add_edge('1','2', 7)
g.add_edge('2','1', 7)
g.add_edge('2','3', 10)
g.add_edge('2','4', 15)
g.add_edge('6','1', 14)
g.add_edge('6','2', 2)
g.add_edge('6','5', 9)
g.add_edge('3','1', 9)
g.add_edge('3','2', 10)
g.add_edge('3','6', 2)
g.add_edge('3','4', 11)
g.add_edge('4','2', 15)
g.add_edge('4','3', 11)
g.add_edge('4','5', 6)
g.add_edge('5','6', 9)
g.add_edge('5','4', 6)

## Алгоритм Дейкстры

In [4]:
class Deijkstra_result_dict:
    def __init__(self, nodes_dict):
        self.nodes_dict = nodes_dict
        
    def get_cost(self, node):
        if self.nodes_dict:
            return self.nodes_dict[node][0]
        
    def get_path(self, node):
        if self.nodes_dict:
            path = []
            path.append(node)
            item = self.nodes_dict[node][1]
            while item:
                path.append(item)
                item = self.nodes_dict[item][1]
            path.reverse()    
            return path
        
    def __repr__(self):
        result = ''
        for key, item in self.nodes_dict.items():
            result = '\n'.join([result, ' : '.join([key, str(item)])])
        return result
        
def get_min_node(nodes_dict):
    minim = np.inf
    result = None
    for key, item in nodes_dict.items():
        if item[2] == 'v':
            continue
        if item[0] < minim:
            minim = item[0]
            result = key
    return result

def compare_edges(graph, start, nodes_dict):
    node = graph.search_node(start)
    edges = node.get_edges()
    start_temp = nodes_dict[node.key]
    for edge in edges:
        temp = nodes_dict[edge.node_in.key]
        if (start_temp[0] + edge.weight) < temp[0]:
            nodes_dict[edge.node_in.key] = [start_temp[0] + edge.weight, start, 'n_v']
    nodes_dict[node.key][2] = 'v'

def deijkctra(graph, node_key):
    nodes_list = graph.nodes_key_list()
    nodes_dict = {}
    for item in nodes_list:
        nodes_dict[item] = [np.inf, None, 'n_v']
    nodes_dict[node_key][0] = 0
    nodes_dict[node_key][1] = 0
    while node_key:      
        compare_edges(graph, node_key, nodes_dict)
        node_key = get_min_node(nodes_dict)
        # метка выхода, если ищем путь только до вершины node_key
    return Deijkstra_result_dict(nodes_dict)    


In [5]:
result = deijkctra(g, '1')

In [6]:
result.get_cost('5')

20

In [7]:
result.get_path('5')

['1', '3', '6', '5']

In [8]:
g2 = Graph()

g2.add_node(Node('A'))
g2.add_node(Node('B'))
g2.add_node(Node('C'))
g2.add_node(Node('D'))
g2.add_node(Node('E'))
g2.add_node(Node('F'))
g2.add_node(Node('G'))


g2.add_edge('A','B', 2)
g2.add_edge('A','C', 3)
g2.add_edge('A','D', 6)
g2.add_edge('B','A', 2)
g2.add_edge('B','C', 4)
g2.add_edge('B','E', 9)
g2.add_edge('C','A', 3)
g2.add_edge('C','B', 4)
g2.add_edge('C','D', 1)
g2.add_edge('C','E', 7)
g2.add_edge('C','F', 6)
g2.add_edge('D','A', 6)
g2.add_edge('D','C', 1)
g2.add_edge('D','F', 4)
g2.add_edge('E','B', 9)
g2.add_edge('E','C', 7)
g2.add_edge('E','F', 1)
g2.add_edge('E','G', 5)
g2.add_edge('F','D', 4)
g2.add_edge('F','C', 6)
g2.add_edge('F','E', 1)
g2.add_edge('F','G', 8)
g2.add_edge('G','E', 5)
g2.add_edge('G','F', 8)

In [9]:
result_2 = deijkctra(g2, 'A')

In [10]:
result_2.get_cost('G')

14

In [11]:
result_2.get_path('G')

['A', 'C', 'D', 'F', 'E', 'G']

In [12]:
result_2


A : [0, 0, 'v']
B : [2, 'A', 'v']
C : [3, 'A', 'v']
D : [4, 'C', 'v']
E : [9, 'F', 'v']
F : [8, 'D', 'v']
G : [14, 'E', 'v']

## Алгоритм Беллмана-Форда

In [13]:
class Result_dict:
    def __init__(self, nodes_dict):
        self.nodes_dict = nodes_dict
        
    def get_cost(self, node):
        if self.nodes_dict:
            return self.nodes_dict[node][0]
        
    def get_path(self, node):
        if self.nodes_dict:
            path = []
            path.append(node)
            item = self.nodes_dict[node][1]
            while item:
                path.append(item)
                item = self.nodes_dict[item][1]
            path.reverse()    
            return path
        
    def __repr__(self):
        result = ''
        for key, item in self.nodes_dict.items():
            result = '\n'.join([result, ' : '.join([key, str(item)])])
        return result

def relax(u, v, w, nodes_dict):
    if nodes_dict[v][0] > (nodes_dict[u][0] + w):
        nodes_dict[v][0] = nodes_dict[u][0] + w
        nodes_dict[v][1] = u
        return True
        
def bellman_ford(graph, start):
    nodes_list = graph.nodes_key_list()
    nodes_dict = {}
    for item in nodes_list:
        nodes_dict[item] = [np.inf, None, 'n_v']
    nodes_dict[start][0] = 0
    nodes_dict[start][1] = 0
    
    edges = []
    for vertex in  graph.nodes_list():
        for edge in vertex:
            edges.append([vertex.key, edge.node_in.key, edge.weight])

## full iteration
#     for _ in range(len(nodes_list) -1):
#         for edge in edges:
#             relax(*edge, nodes_dict) 
   
## iterations, while change
    count = 0
    border = len(nodes_list) -1
    while True:
        check_change = False
        for edge in edges:            
            if relax(*edge, nodes_dict):
                check_change = True
        if not check_change:
            break
        count += 1
        if count > border:
            print('negative cycle')
            print(Result_dict(nodes_dict))
            return 
            
    return Result_dict(nodes_dict)
        

In [14]:
result = bellman_ford(g2, 'A')

In [15]:
result.get_path('G')

['A', 'C', 'D', 'F', 'E', 'G']

In [16]:
result.get_cost('G')

14

In [17]:
result = bellman_ford(g, '1')
result.get_path('5')

['1', '3', '6', '5']

In [18]:
result.get_cost('5')

20

In [19]:
# 'negative cycle'
g3 = Graph()

g3.add_node(Node('A'))
g3.add_node(Node('B'))
g3.add_node(Node('C'))
g3.add_node(Node('D'))
g3.add_node(Node('E'))
g3.add_node(Node('F'))
g3.add_node(Node('G'))


g3.add_edge('A','B', 2)
g3.add_edge('A','C', 3)
g3.add_edge('A','D', 6)
g3.add_edge('B','A', 2)
g3.add_edge('B','C', 4)
g2.add_edge('B','E', 9)
g3.add_edge('C','A', 3)
g3.add_edge('C','B', 4)
g3.add_edge('C','D', -10)
g3.add_edge('C','E', 7)
g3.add_edge('C','F', 6)
g3.add_edge('D','A', 6)
g3.add_edge('D','C', -10)
g3.add_edge('D','F', 4)
g3.add_edge('E','B', 9)
g3.add_edge('E','C', 7)
g3.add_edge('E','F', 1)
g3.add_edge('E','G', 5)
g3.add_edge('F','D', 4)
g3.add_edge('F','C', 6)
g3.add_edge('F','E', 1)
g3.add_edge('F','G', 8)
g3.add_edge('G','E', 5)
g3.add_edge('G','F', 8)

In [20]:
result = bellman_ford(g3, 'A')

negative cycle

A : [-121, 'D', 'n_v']
B : [-113, 'C', 'n_v']
C : [-137, 'D', 'n_v']
D : [-127, 'C', 'n_v']
E : [-122, 'F', 'n_v']
F : [-123, 'D', 'n_v']
G : [-115, 'F', 'n_v']


In [21]:
# виден цикл - 'C'-'D'-'F'